# Cluster Photos

This notebook leverages learned features and out-of-the-box clustering techniques to cluster photos with the intent of finding a representative sample.

## Instantiate Autoencoder

Let's create our autoencoder and do preliminary work to process our images.

In [12]:
from photo_encode.encoder import FaissIndexImageEncoder
from transformers import AutoImageProcessor, AutoModel, ViTImageProcessor, ViTForImageClassification

# First embedding model: dinov2
processor_dino = AutoImageProcessor.from_pretrained('facebook/dinov2-large')
dinov2_hf = AutoModel.from_pretrained('facebook/dinov2-large')
dinov2 = FaissIndexImageEncoder(dinov2_hf, index_file_out="./data/test_photos_dinov2.db").eval()

# Second embedding model: ViT
processor_vit = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
vit_hf = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
vit = FaissIndexImageEncoder(vit_hf, index_file_out="./data/test_photos_vit.db").eval()

## Create the Index

Run the embedding models over the imagery to create the vector database.

In [14]:
import torch

from photo_encode.utils import image_batch_from_folder_generator
from torchvision.transforms import Resize, Compose

# Directory with photos
photo_directory = "./data/test_photos"

# Batch size
batch_size = 8

# Walk over the photo directory
with torch.no_grad():
    for x, files in image_batch_from_folder_generator(photo_directory, processor_dino, batch_size):
        print(x, files)

# Flush the embeddings to the database